In [156]:
import internetarchive
import os
import tarfile
import csv
from datetime import datetime
from tqdm.notebook import tqdm
import pandas as pd

In [186]:
def control_op(control_file):

    #if file doesn't exist I'll create it
    if not os.path.exists(control_file):
        with open(control_file, 'w', newline='') as control_csv:
            writer = csv.DictWriter(control_csv, fieldnames=['name','datetime'])
            writer.writeheader()
        control_csv.close()
        print("{} created.".format(control_file))
            
    #Reading the control file
    with open(control_file, 'r', newline='') as control_csv:
        reader = csv.DictReader(control_csv)
        reader_data = [r for r in reader]
        control_csv.seek(0)
    control_csv.close()
    
    return reader_data

In [222]:
def check_folder(name_folder):            
    #Creating folder if that doesn't exist
    p = %pwd
    p = p + f'/{name_folder}'
    path = os.path.expanduser(p)
    if not os.path.exists(path):
        os.makedirs(path)
        print("{} created.".format(path))
        
    return path

In [203]:
def get_file_name(item_name, ext='*tar'):
    
    file_names = [f.name for f in internetarchive.get_files(item_name, glob_pattern= ext)]
    
    return file_names

In [228]:
def download_file(item_name, file_name, download_folder = 'Downloads'):
    
    path = check_folder(download_folder)
    
    #calling function for getting control file
    control_path = path + '/control_download.csv'    
    reader_data = control_op(control_path)
    
    #Getteing Itens names
    item = internetarchive.get_item(item_name)
        
    #openning control file to add file names on the list
    with open(control_path, 'a', newline='') as control_download:
        writer = csv.DictWriter(control_download, fieldnames=['name','datetime'])
        
        #check if the file has been downloaded before
        if not any(row['name'] == file_name for row in reader_data):
            
            #downloading
            r = item.download(
                destdir=path,  # The directory to download files to
                ignore_existing=True,  # Skip files that already exist locally
                checksum=True,  # Skip files based on checksum
                verbose=True,  # Print progress to stdout
                retries=100,  # The number of times to retry on failed requests
                no_directory=True,  # Download withtout the identifier
                files = file_name)
            
            #Adding file name on control list
            row = {'name' : t, 'datetime': datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
            writer.writerow(row)
            
    control_download.close()
    
    path_file = path + f'/{file_name}'
    
    return path_file

In [343]:
def tar_file(tar_file, file_name, extration_folder='Extraction'):

    path = check_folder(extration_folder)
    
    tar = tar_file
    
    #calling function for getting control file
    control_path = path + '/control_extraction.csv'    
    reader_data = control_op(control_path)

    #openning control file to add file names on the list
    with open(control_path, 'a', newline='') as control_tar:
        writer = csv.DictWriter(control_tar, fieldnames=['name','datetime'])
        
        #check if the file has been extrated before
        if not any(row['name'] == file_name for row in reader_data):

            #Extracting file
            tar.extract(file_name, path=path)

            #Saving files name on control file
            row = {'name' : file_name, 'datetime': datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
            writer.writerow(row)
            
            path_file = path + f'/{file_name}'
            
        else:
            
            path_file = False
    
    control_tar.close()
    
    
    return path_file

In [310]:
def get_data(file_path, word_bag = False, lang = False):
    
    #reading json
    df = pd.read_json(file_path, lines=True, compression='gzip')
    
    #Filtering language if a language was sent
    if not lang == False:
        df = df[df.lang == lang]
    
    #taking away columns unnecessary
    df = df.loc[:, ['created_at', 'text', 'entities']]
    
    #filtering Tweets using subject word bag if a word bag was sent
    if not word_bag == False:
        bow = '|'.join(word_bag) # bag of word
        df = df[df['text'].str.contains(bow, case=False)]
    
    return df

In [341]:
#dataframe to store the tweets
tweets = pd.DataFrame({'created_at': [], 'text': [], 'entities': []})

In [342]:
item_name = 'archiveteam-twitter-stream-2022-10'

#Getting Itens names
file_names = get_file_name(item_name = item_name)#, ext = '*jpg')

x = 0 #for testing 

for dlf in tqdm(file_names, desc='Download progress'):
    
    if x < 1: #for testing 
        #downloading the file
        file = download_file(item_name = item_name, file_name = dlf)

        #Open tar file
        tar = tarfile.open(file, "r")

        #Getting json files' names
        tar_file_names = list(filter(lambda t: t.endswith('.json.gz'), tar.getnames()))

        for tf in tqdm(tar_file_names, desc='Unzip progress'):
            js = tar_file(tar_file = tar, file_name = tf)
            
            #Case file has already been unzipped
            if not js == False:
            
                #Getting, cleaning and storing Data
                bow = ['vaccination', 'vaccines', 'vaccine']
                df = get_data(file_path = js, lang = 'en', word_bag = bow)
                if len(df) > 0:
                    tweets = pd.concat([tweets,df], ignore_index=True)

                #deleting json file after processing
                os.remove(js)

        tar.close()
        
        x =+ 1

Download progress:   0%|          | 0/31 [00:00<?, ?it/s]

archiveteam-twitter-stream-2022-10:
 skipping /home/hduser/Desktop/CA 2.2/Downloads/twitter-stream-20221001.tar, file already exists.


Unzip progress:   0%|          | 0/1421 [00:00<?, ?it/s]

/home/hduser/Desktop/CA 2.2/Extration created.
/home/hduser/Desktop/CA 2.2/Extration/control_extraction.csv created.


In [344]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815 entries, 0 to 814
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   created_at  815 non-null    datetime64[ns, UTC]
 1   text        815 non-null    object             
 2   entities    815 non-null    object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 19.2+ KB
